In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import re

In [2]:
df_old = pd.read_csv(r'C:\Users\DELL\Documents\Code\mitwpu-team_4-currency_exchange\conversion_rates\Exchange_Rate_Report_2022.csv')
new_cols = []
for x in list(df_old.columns):
    new_cols.append(x.strip())

# List to hold extracted currency codes
currency_codes = ['Date']

# Regex pattern to extract the currency code
pattern = r'\((\w{3})\)'

# Loop through each country name and extract the code
for country in new_cols:
    match = re.search(pattern, country)
    if match:
        currency_code = match.group(1)
        currency_codes.append(currency_code)

# Output the list of currency codes
print("Extracted Currency Codes:", currency_codes)

Extracted Currency Codes: ['Date', 'DZD', 'AUD', 'BWP', 'BRL', 'BND', 'CAD', 'CLP', 'CNY', 'CZK', 'DKK', 'EUR', 'INR', 'ILS', 'JPY', 'KRW', 'KWD', 'MYR', 'MUR', 'MXN', 'NZD', 'NOK', 'OMR', 'PEN', 'PHP', 'PLN', 'QAR', 'RUB', 'SAR', 'SGD', 'ZAR', 'SEK', 'CHF', 'THB', 'TTD', 'AED', 'GBP', 'USD', 'UYU']


In [3]:
df_old.head(2)

,Date,Algerian dinar (DZD),Australian dollar (AUD),Botswana pula (BWP),Brazilian real (BRL),Brunei dollar (BND),Canadian dollar (CAD),Chilean peso (CLP),Chinese yuan (CNY),Czech koruna (CZK),...,Singapore dollar (SGD),South African rand (ZAR),Swedish krona (SEK),Swiss franc (CHF),Thai baht (THB),Trinidadian dollar (TTD),U.A.E. dirham (AED),U.K. pound (GBP),U.S. dollar (USD),Uruguayan peso (UYU)
0,3-Jan-22,138.6713,NaN,NaN,5.6303,1.3510,NaN,844.69,NaN,21.860,...,1.3510,15.82355,9.0504,0.9121,NaN,6.7742,3.6725,NaN,1,44.614
1,4-Jan-22,138.9115,1.38677,NaN,5.6770,1.3548,1.2708,851.43,6.3751,21.944,...,1.3548,16.04260,9.0956,0.9187,33.311,6.7723,3.6725,0.740988,1,44.725


In [4]:
df = pd.read_csv(r'C:\Users\DELL\Documents\Code\mitwpu-team_4-currency_exchange\conversion_rates\Exchange_Rate_Report_2022.csv', names=currency_codes, header=0)
df = df.replace({np.nan: None})
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y').dt.strftime('%Y-%m-%d')

In [5]:
df.head(2)

,Date,DZD,AUD,BWP,BRL,BND,CAD,CLP,CNY,CZK,...,SGD,ZAR,SEK,CHF,THB,TTD,AED,GBP,USD,UYU
0,2022-01-03,138.6713,None,None,5.6303,1.351,None,844.69,None,21.86,...,1.351,15.82355,9.0504,0.9121,None,6.7742,3.6725,None,1,44.614
1,2022-01-04,138.9115,1.38677,None,5.677,1.3548,1.2708,851.43,6.3751,21.944,...,1.3548,16.0426,9.0956,0.9187,33.311,6.7723,3.6725,0.740988,1,44.725


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 39 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    165 non-null    object
 1   DZD     155 non-null    object
 2   AUD     156 non-null    object
 3   BWP     148 non-null    object
 4   BRL     149 non-null    object
 5   BND     147 non-null    object
 6   CAD     149 non-null    object
 7   CLP     159 non-null    object
 8   CNY     153 non-null    object
 9   CZK     158 non-null    object
 10  DKK     152 non-null    object
 11  EUR     163 non-null    object
 12  INR     152 non-null    object
 13  ILS     150 non-null    object
 14  JPY     155 non-null    object
 15  KRW     145 non-null    object
 16  KWD     119 non-null    object
 17  MYR     150 non-null    object
 18  MUR     156 non-null    object
 19  MXN     151 non-null    object
 20  NZD     154 non-null    object
 21  NOK     148 non-null    object
 22  OMR     121 non-null    ob

In [7]:
# Connect to MySQL Database
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password=''
)

cursor = connection.cursor()
database_name = 'currency_data22'
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")
cursor.execute(f"USE {database_name}")
# Create the table with the appropriate schema

In [8]:
table_creation = "CREATE TABLE IF NOT EXISTS currency_rates22 ( Date DATE"
for x in currency_codes[1:]:
    table_creation += f", {x} DECIMAL(10,4)"
table_creation += ");"
cursor.execute(table_creation)

In [9]:
table_creation

'CREATE TABLE IF NOT EXISTS currency_rates22 ( Date DATE, DZD DECIMAL(10,4), AUD DECIMAL(10,4), BWP DECIMAL(10,4), BRL DECIMAL(10,4), BND DECIMAL(10,4), CAD DECIMAL(10,4), CLP DECIMAL(10,4), CNY DECIMAL(10,4), CZK DECIMAL(10,4), DKK DECIMAL(10,4), EUR DECIMAL(10,4), INR DECIMAL(10,4), ILS DECIMAL(10,4), JPY DECIMAL(10,4), KRW DECIMAL(10,4), KWD DECIMAL(10,4), MYR DECIMAL(10,4), MUR DECIMAL(10,4), MXN DECIMAL(10,4), NZD DECIMAL(10,4), NOK DECIMAL(10,4), OMR DECIMAL(10,4), PEN DECIMAL(10,4), PHP DECIMAL(10,4), PLN DECIMAL(10,4), QAR DECIMAL(10,4), RUB DECIMAL(10,4), SAR DECIMAL(10,4), SGD DECIMAL(10,4), ZAR DECIMAL(10,4), SEK DECIMAL(10,4), CHF DECIMAL(10,4), THB DECIMAL(10,4), TTD DECIMAL(10,4), AED DECIMAL(10,4), GBP DECIMAL(10,4), USD DECIMAL(10,4), UYU DECIMAL(10,4));'

In [14]:
insertion_format = "INSERT INTO currency_rates22 (Date "
for x in currency_codes[1:]:
    insertion_format += f", {x}"
insertion_format += ") VALUES"
insertion_format

'INSERT INTO currency_rates22 (Date , DZD, AUD, BWP, BRL, BND, CAD, CLP, CNY, CZK, DKK, EUR, INR, ILS, JPY, KRW, KWD, MYR, MUR, MXN, NZD, NOK, OMR, PEN, PHP, PLN, QAR, RUB, SAR, SGD, ZAR, SEK, CHF, THB, TTD, AED, GBP, USD, UYU) VALUES'

In [27]:
for idx, row in df.iterrows():
    print(row.values)
    toInsert = "" + insertion_format + "(" + f"\"{str(row.values[0])}\""
    for field in row.values[1:]:
        if field is None:
            toInsert += f", NULL"
        else:
            toInsert += f", {field}"
    toInsert += ");"
    print(toInsert)
    cursor.execute(toInsert)
    connection.commit()
    break

['2022-01-03' 138.6713 None None 5.6303 1.351 None 844.69 None 21.86
 6.5506 0.880669309 74.3115 3.092 None 1185.5 0.3025 4.171 None 20.589
 None 8.8078 0.3845 3.987 50.974 4.0424 3.64 None 3.75 1.351 15.82355
 9.0504 0.9121 None 6.7742 3.6725 None 1 44.614]
INSERT INTO currency_rates22 (Date , DZD, AUD, BWP, BRL, BND, CAD, CLP, CNY, CZK, DKK, EUR, INR, ILS, JPY, KRW, KWD, MYR, MUR, MXN, NZD, NOK, OMR, PEN, PHP, PLN, QAR, RUB, SAR, SGD, ZAR, SEK, CHF, THB, TTD, AED, GBP, USD, UYU) VALUES("2022-01-03", 138.6713, NULL, NULL, 5.6303, 1.351, NULL, 844.69, NULL, 21.86, 6.5506, 0.880669309, 74.3115, 3.092, NULL, 1185.5, 0.3025, 4.171, NULL, 20.589, NULL, 8.8078, 0.3845, 3.987, 50.974, 4.0424, 3.64, NULL, 3.75, 1.351, 15.82355, 9.0504, 0.9121, NULL, 6.7742, 3.6725, NULL, 1, 44.614);


InternalError: Unread result found

In [24]:
print(cursor.execute("SHOW DATABASES"))

InternalError: Unread result found